In [ ]:
%%script bash

# Remove `/sample_data` & create our folders
rm -rf sample_data

# Part B 

## Part 1. Sequential


In [ ]:
%%writefile grid.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define p 0.5
#define n 0.0002
#define G 0.75
#define N 4

void sequential(float u[N * N], float u1[N * N], float u2[N * N])
{
  // Iterate through the center of the grid and apply formula
  for (int i = 1; i < N - 1; i++){
    for (int j = 1; j < N - 1; j++){
      float a =
        u1[(i - 1) * N + j] 
        + u1[(i + 1) * N + j] 
        + u1[i * N + (j - 1)] 
        + u1[i * N + (j + 1)] 
        - 4 * u1[i * N + j];

      float b = 2 * u1[i * N + j] - (1 - n) * u2[i * N + j];

      u[i * N + j] = (p * a + b) / (1 + n);
    }
  }

  // Iterate through side elements
	for (int i = 1; i < N-1; i++) {
    u[0 + i] = G * u[1 * N + i];
    u[(N - 1) * N + i] = G * u[(N - 2) * N + i];
    u[(i * N) + 0] = G * u[i * N + 1];
    u[(i * N) + (N - 1)] = G * u[i * N + (N - 2)];
  }

  // Corners
  u[0 + 0] = G * u[1 * N + 0];
  u[(N - 1) * N + 0] = G * u[(N - 2) * N + 0];
  u[0 + (N - 1)] = G * u[0 + (N - 2)];
  u[(N - 1) * N + (N - 1)] = G * u[(N - 1) * N + (N - 2)];

  // Set u2 = u1 and u1 = u
  for (int i = 0; i < N * N; i++){
    u2[i] = u1[i];
    u1[i] = u[i];
  }
}

int main(int argc, char* argv[])
{
  if (argc != 2){
    printf("Usage: ./sequential <number_of_iterations>\n");
    exit(1);
  }

	int iterations = atoi(argv[1]);
	printf("Processing 4x4 grid with %d iterations...\n", iterations);

	float* u = (float*)malloc(N * N * sizeof(float));
	float* u1 = (float*)malloc(N * N * sizeof(float));
	float* u2 = (float*)malloc(N * N * sizeof(float));
 
	for (int i = 0; i < N * N; i++) {
		u[i] = 0.f;
		u1[i] = 0.f;
		u2[i] = 0.f;
	}

  for (int i = 0; i < iterations; i++){
    if (i == 0)
      u1[(N / 2) * N + (N / 2)] = 1;

    sequential(u, u1, u2);
    printf("\n(%d,%d): ", N/2, N / 2);
    printf("%.6f\n", u[(N / 2) * N + (N / 2)]);
  }
  
  cudaFree(u);
  cudaFree(u1);
  cudaFree(u2);
 
  return 0;
}

Writing grid.cu


In [ ]:
%%script bash
nvcc grid.cu  -o sequential

iterations=3

./sequential $iterations

Processing 4x4 grid with 3 iterations...

(2,2): 0.000000

(2,2): -0.499800

(2,2): 0.000000


## Part 2: Parallel ./grid_4_4

In [ ]:
%%writefile parallel_grid.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define p 0.5
#define n 0.0002
#define G 0.75
#define N 4
#define MAX_THREAD 1020

__global__
void process_center(float u[N * N], float u1[N * N], float u2[N * N])
{
  int center_length = N-2;
  int index = blockIdx.x * blockDim.x + threadIdx.x;
 
  if(index < center_length * center_length) {
    int j = index  % center_length;
    int i = (index - j + 1)/ center_length + 1 ;
    j = j + 1;

    float a =
      u1[(i - 1) * N + j] 
      + u1[(i + 1) * N + j] 
      + u1[i * N + (j - 1)] 
      + u1[i * N + (j + 1)] 
      - 4 * u1[i * N + j];

    float b = 2 * u1[i * N + j] - (1 - n) * u2[i * N + j];

    u[i * N + j] = (p * a + b) / (1 + n);
  }
}

__global__
void process_sides(float u[N * N])
{
  int center_length = N-2;
 
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int i = index % center_length + 1; 

  // TOP  
  if (index < center_length){
    u[0 + i] = G * u[1 * N + i];
  }
  // BOTTOM 
  else if (index < (2 * center_length)){
    u[(N - 1) * N + i] = G * u[(N - 2) * N + i];
  } 
  // LEFT
  else if (index < (3 * center_length)){
    u[(i * N) + 0] = G * u[i * N + 1];
  }
  // RIGHT
  else if (index < (4 * center_length)){
    u[(i * N) + (N - 1)] = G * u[i * N + (N - 2)];
  }
}

void process_corners(float u[N * N])
{
  // Corners
  u[0 + 0] = G * u[1 * N + 0];
  u[(N - 1) * N + 0] = G * u[(N - 2) * N + 0];
  u[0 + (N - 1)] = G * u[0 + (N - 2)];
  u[(N - 1) * N + (N - 1)] = G * u[(N - 1) * N + (N - 2)];
}

void update(float u[N * N], float u1[N * N], float u2[N * N])
{
  // Set u2 = u1 and u1 = u
  for (int i = 0; i < N * N; i++){
    u2[i] = u1[i];
    u1[i] = u[i];
  }
}

int main(int argc, char* argv[])
{
  if (argc != 2){
    printf("Usage: ./grid_4_4 <number_of_iterations>\n");
    exit(1);
  }

	int iterations = atoi(argv[1]);
	printf("Processing %dx%d grid with %d iterations...\n", N, N, iterations);

	float *u, *u1, *u2;
  
  cudaMallocManaged(&u, N * N * sizeof(float));
  cudaMallocManaged(&u1, N * N * sizeof(float));
  cudaMallocManaged(&u2, N * N * sizeof(float));

	for (int i = 0; i < N * N; i++) {
		u[i] = 0.f;
		u1[i] = 0.f;
		u2[i] = 0.f;
	}
  
  int num_block_center = 1;
  int num_block_sides = 1;
  int num_thread_center = (N-2) * (N-2);
  int num_thread_sides = (N-2) * 4;
  
  if (num_thread_center > MAX_THREAD){
    num_block_center = num_thread_center / MAX_THREAD + 1;
    num_thread_center = MAX_THREAD;
  }
  if (num_thread_sides > MAX_THREAD){
    num_block_sides = num_thread_sides / MAX_THREAD + 1;
    num_thread_sides = MAX_THREAD;
  }
  
  for (int i = 0; i < iterations; i++){
      
    if (i == 0)
      u1[(N / 2) * N + (N / 2)] = 1;

    process_center <<< num_block_center, num_thread_center >>> (u, u1, u2);
    cudaDeviceSynchronize();

    process_sides <<< num_block_sides, num_thread_sides >>> (u);
    cudaDeviceSynchronize();

    process_corners(u);
    update(u, u1, u2);

    printf("\n(%d,%d): ", N/2, N / 2);
    printf("%.6f\n", u[(N / 2) * N + (N / 2)]);
  }
 
  cudaFree(u);
  cudaFree(u1);
  cudaFree(u2);
 
  return 0;
}

Overwriting parallel_grid.cu


In [ ]:
%%script bash
nvcc parallel_grid.cu  -o grid_4_4

iterations=3

./grid_4_4 $iterations

Processing 4x4 grid with 3 iterations...

(2,2): 0.000000

(2,2): -0.499800

(2,2): 0.000000


## Part 3: Decomposition ./grid_512_512

In [ ]:
%%writefile decomposition_grid.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define p 0.5
#define n 0.0002
#define G 0.75
#define N 512
#define MAX_THREAD 1020
#define ELEMENTS_PER_THREAD 5

__global__
void process_center(float u[N * N], float u1[N * N], float u2[N * N])
{
  int center_length = N-2;
  int index = (blockIdx.x * blockDim.x + threadIdx.x) * ELEMENTS_PER_THREAD;
 
  if(index < center_length * center_length) {  
    for(int x = 0; x < ELEMENTS_PER_THREAD; x++){
      int j = index  % center_length;
      int i = (index - j + 1)/ center_length + 1 ;
      j = j + 1;

      float a =
        u1[(i - 1) * N + j] 
        + u1[(i + 1) * N + j] 
        + u1[i * N + (j - 1)] 
        + u1[i * N + (j + 1)] 
        - 4 * u1[i * N + j];

      float b = 2 * u1[i * N + j] - (1 - n) * u2[i * N + j];

      u[i * N + j] = (p * a + b) / (1 + n);

      index++;
    }    
  }
}

__global__
void process_sides(float u[N * N])
{
  int center_length = N-2;
 
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int i = index % center_length + 1; 

  // TOP  
  if (index < center_length){
    u[0 + i] = G * u[1 * N + i];
  }
  // BOTTOM 
  else if (index < (2 * center_length)){
    u[(N - 1) * N + i] = G * u[(N - 2) * N + i];
  } 
  // LEFT
  else if (index < (3 * center_length)){
    u[(i * N) + 0] = G * u[i * N + 1];
  }
  // RIGHT
  else if (index < (4 * center_length)){
    u[(i * N) + (N - 1)] = G * u[i * N + (N - 2)];
  }
}

void process_corners(float u[N * N])
{
  // Corners
  u[0 + 0] = G * u[1 * N + 0];
  u[(N - 1) * N + 0] = G * u[(N - 2) * N + 0];
  u[0 + (N - 1)] = G * u[0 + (N - 2)];
  u[(N - 1) * N + (N - 1)] = G * u[(N - 1) * N + (N - 2)];
}

void update(float u[N * N], float u1[N * N], float u2[N * N])
{
  // Set u2 = u1 and u1 = u
  for (int i = 0; i < N * N; i++){
    u2[i] = u1[i];
    u1[i] = u[i];
  }
}

int main(int argc, char* argv[])
{
  if (argc != 2){
    printf("Usage: ./sequential <number_of_iterations>\n");
    exit(1);
  }

	int iterations = atoi(argv[1]);
	printf("Processing %dx%d grid with %d iterations...\n", N, N, iterations);

	float *u, *u1, *u2;  
  cudaMallocManaged(&u, N * N * sizeof(float));
  cudaMallocManaged(&u1, N * N * sizeof(float));
  cudaMallocManaged(&u2, N * N * sizeof(float));

	for (int i = 0; i < N * N; i++) {
		u[i] = 0.f;
		u1[i] = 0.f;
		u2[i] = 0.f;
	}

  int num_block_center = 1;
  int num_block_sides = 1;
  int num_thread_center = ((N-2) * (N-2)) / ELEMENTS_PER_THREAD;
  int num_thread_sides = (N-2) * 4;
  
  if (num_thread_center > MAX_THREAD){
    num_block_center = num_thread_center / MAX_THREAD + 1;
    num_thread_center = MAX_THREAD;
  }
  if (num_thread_sides > MAX_THREAD){
    num_block_sides = num_thread_sides / MAX_THREAD + 1;
    num_thread_sides = MAX_THREAD;
  }
 
  // Timer starts
	float time;
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0); 
  
  for (int i = 0; i < iterations; i++){
    if (i == 0)
      u1[(N / 2) * N + (N / 2)] = 1;

    process_center <<< num_block_center, num_thread_center >>> (u, u1, u2);
    cudaDeviceSynchronize();

    process_sides <<< num_block_sides, num_thread_sides>>> (u);
    cudaDeviceSynchronize();

    process_corners(u);
    update(u, u1, u2);

    printf("\n(%d,%d): ", N/2, N / 2);
    printf("%.6f\n", u[(N / 2) * N + (N / 2)]);
  }
 
  // Timer stops
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&time, start, stop);
  
  cudaFree(u);
  cudaFree(u1);
  cudaFree(u2);
 
  return 0;
}

Overwriting decomposition_grid.cu


In [ ]:
%%script bash
nvcc decomposition_grid.cu  -o grid_512_512

iterations=12

./grid_512_512 $iterations

Processing 512x512 grid with 12 iterations...

(256,256): 0.000000

(256,256): 0.000000

(256,256): 0.000000

(256,256): 0.249800

(256,256): 0.000000

(256,256): 0.000000

(256,256): 0.000000

(256,256): 0.140400

(256,256): 0.000000

(256,256): 0.000000

(256,256): 0.000000

(256,256): 0.097422
